In [1]:
from pyjet import cluster,DTYPE_EP,DTYPE_PTEPM

In [13]:
import sys
import optparse
import os
import glob
import matplotlib.pyplot as plt
import pandas as pd
#import root_pandas as rpd
import uproot3
#import awkward as ak
import numpy as np
import math
#timers
import time
#if args.time:
#    start = time.clock()
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)

In [20]:
tree = uproot3.open("ntuple_2.root",xrootdsource=dict(chunkbytes=1024**3, limitbytes=1024**3))["FloatingpointThresholdDummyHistomaxGenmatchGenclustersntuple/HGCalTriggerNtuple"]
full = ["genjet_eta","genjet_phi","genjet_pt","genjet_energy","cl3d_pt","cl3d_energy","cl3d_eta","cl3d_phi"]
branches = ["genjet_eta_*","genjet_phi_*","genjet_pt_*","genjet_energy_*","cl3d_pt","cl3d_energy","cl3d_eta","cl3d_phi"]
branches2 = ["genjet_eta","genjet_phi","genjet_pt","genjet_energy"]

IOError: [Errno 2] No such file or directory: 'ntuple_2.root'

In [15]:
fulldf = tree.pandas.df(full,flatten=False)
df = tree.pandas.df(branches,entrystart=0,entrystop=tree.numentries)
df2 = tree.pandas.df(branches2,entrystart=0,entrystop=tree.numentries)
fulldf["ak4jet_eta"] = fulldf["cl3d_eta"]
fulldf["ak4jet_phi"] = fulldf["cl3d_phi"]
fulldf["ak4jet_pt"] = fulldf["cl3d_pt"]
fulldf["ak4jet_constituents"] = fulldf["cl3d_energy"]
#fulldf["deta"] = fulldf["cl3d_eta"]
#fulldf["dphi"] = fulldf["cl3d_phi"]
#fulldf["dpt"] = fulldf["cl3d_pt"]
#fulldf["deltar"] = fulldf["cl3d_eta"]
#fulldf

In [16]:
for i_event in range(tree.numentries):   
    genDF = df.loc[i_event]
    genjet = df2.loc[i_event,['genjet_pt','genjet_eta','genjet_phi','genjet_energy']]
    #genjet.columns=["pT","eta","phi","energy"]
    cl3d = df.loc[i_event,['cl3d_pt','cl3d_eta','cl3d_phi','cl3d_energy']]
    cl3d.columns=["pT","eta","phi","energy"]
    cl3d = cl3d.assign(mass=0)
    #go dataframe to np array, formatted for input into fastjet    
    cl3dVectors = np.array(cl3d[["pT","eta","phi","energy"]].to_records(index=False).astype([(u'pT', '<f8'), (u'eta', '<f8'), (u'phi', '<f8'), (u'mass', '<f8')]) )    
    clusterVals = cluster(cl3dVectors,R=0.4,algo="antikt")
    _jets = clusterVals.inclusive_jets() 
    ak4jet_eta = []
    ak4jet_phi = []
    ak4jet_pt = []
    constituent = []
    deta = []
    dphi = []
    deltar = []
    dpt = []

    for j,jet in enumerate(_jets):
        ak4jet_eta.append(jet.eta)
        ak4jet_phi.append(jet.phi)
        ak4jet_pt.append(jet.pt)
        constituent.append((len(_jets[j].constituents_array())))
        
#        deta.append(jet.eta-genjet['genjet_eta'])
#        dphi.append((abs(abs(jet.phi-genjet['genjet_phi'])-np.pi)-np.pi))
#        deltar.append(((jet.eta-genjet['genjet_eta'])**2 + ((abs(abs(jet.phi-genjet['genjet_phi'])-np.pi)-np.pi))**2)**0.5)      
#        dpt.append(jet.pt-genjet['genjet_pt'])
    
#    for k in range (0,len(deltar[i_event])):
#        if deltar[i_event][k]<0.2:
#            print (deltar[i_event][k])

    fulldf['ak4jet_eta'][i_event] = ak4jet_eta
    fulldf['ak4jet_phi'][i_event] = ak4jet_phi
    fulldf['ak4jet_pt'][i_event] = ak4jet_pt
    fulldf['ak4jet_constituents'][i_event] = constituent
#    fulldf['deta'][i_event] = deta
#    fulldf['dphi'][i_event] = dphi
#    fulldf['dpt'][i_event] = dpt
#    fulldf['deltar'][i_event] = deltar
    
    
    
    del ak4jet_eta
    del ak4jet_phi
    del ak4jet_pt
    del constituent
#    del deta
#    del dphi
#    del dpt
#    del deltar
    
fulldf
#print(constituent)


,genjet_eta,genjet_phi,genjet_pt,genjet_energy,cl3d_pt,cl3d_energy,cl3d_eta,cl3d_phi,ak4jet_eta,ak4jet_phi,ak4jet_pt,ak4jet_constituents
entry,,,,,,,,,,,,
0,"[3.661714, -2.4863205, -0.4991139, -4.1430526,...","[0.94959605, 2.0183802, -0.3163363, 1.2824774,...","[82.675644, 65.43992, 13.722247, 8.944778, 8.5...","[1610.2673, 396.16376, 15.737003, 281.82437, 9...","[2.03097, 5.3105865, 4.177999, 2.4996977, 5.33...","[21.257528, 56.991215, 43.07331, 28.040743, 57...","[-3.0390549, -3.0641642, -3.0238578, -3.108643...","[-2.5185525, -2.4227152, -2.0366101, -1.967131...","[-2.53838736887, -2.88518266993, -2.8990910223...","[2.06604920194, -0.887338139038, 0.18581094284...","[61.5661853212, 42.841266578, 24.6713569373, 2...","[5, 5, 3, 5, 3, 3, 2, 2, 2, 3, 2, 2, 2, 1]"
1,"[-1.0106914, 4.1444936, 4.6383033, -5.3820314,...","[2.372956, 2.8036356, -0.36368093, -1.0623943,...","[53.71416, 16.370287, 13.973904, 12.145084, 10...","[84.101814, 516.5138, 722.3252, 1320.594, 238....","[3.8312511, 8.157053, 7.469509, 4.862891, 2.63...","[40.718952, 79.13814, 69.49664, 43.773003, 23....","[-3.0544288, -2.9627924, -2.9206958, -2.887431...","[-2.4064908, -2.978832, -0.75232476, -0.588869...","[2.74041768269, -2.96240187443, -2.81749413171...","[-2.10002828143, 1.26324434414, 2.85001242581,...","[28.1487298587, 23.0226971886, 22.7560967028, ...","[4, 3, 5, 3, 5, 3, 4, 1, 2, 2, 1, 1, 1, 1]"
2,"[-1.7732373, -1.0899273, 3.6337605, 4.733906, ...","[2.217401, -3.0570362, 2.983815, -0.21631227, ...","[61.11456, 52.461407, 27.268953, 21.477528, 8....","[185.49518, 87.0164, 516.5062, 1221.5143, 21.5...","[2.9885132, 5.1107097, 4.748715, 7.397834, 6.9...","[31.53537, 55.178646, 54.56306, 78.37493, 66.7...","[-3.047228, -3.0702333, -3.1327312, -3.0512292...","[-0.68406755, -0.5757261, 1.9452745, 2.0862029...","[-2.21840951792, -2.82199266072, 2.94983213776...","[2.34138704696, 1.82194215989, -1.74039084361,...","[52.1650257716, 35.2299055304, 33.4573239839, ...","[6, 5, 7, 3, 3, 2, 2, 1, 2, 1, 1, 1, 1]"
3,"[-2.279032, 2.845848, 4.559407, 2.131759, 6.24...","[1.0248489, -1.7532537, -2.0303216, -2.876917,...","[87.56582, 6.364548, 6.31827, 4.730365, 4.6471...","[432.2802, 55.036404, 301.83304, 20.446812, 12...","[8.66264, 9.0012, 5.02312, 2.247446, 3.7140584...","[73.29851, 71.78801, 36.085896, 14.400227, 22....","[-2.8251576, -2.765553, -2.6601183, -2.5444508...","[0.8219242, 1.3491013, 2.2920775, 2.2155666, -...","[-2.40275569789, 2.73594288805, 2.98306750017,...","[1.03842773887, 2.44418175357, -1.70302242847,...","[86.2817196298, 38.5732942926, 30.8933911459, ...","[4, 6, 4, 4, 4, 4, 3, 2, 3, 1, 1]"
4,"[2.917549, -4.0767636, -3.5512078, 0.95628256,...","[-0.22523353, 0.07979044, -1.5074965, -0.17130...","[86.99222, 69.0748, 29.13906, 10.037156, 6.811...","[807.0572, 2036.7283, 508.28845, 15.161914, 72...","[10.441444, 5.160666, 6.986469, 3.4502087, 5.1...","[94.97195, 46.50742, 62.886486, 32.45282, 49.1...","[-2.89791, -2.8886008, -2.8874032, -2.931662, ...","[-2.9739008, -2.4044366, -1.6522495, -1.557675...","[2.96954081028, 2.92360614979, 2.80818694955, ...","[-0.20131631139, -2.96624805007, -2.1205365036...","[86.9840457686, 52.814495846, 46.5511265215, 4...","[5, 8, 9, 5, 5, 5, 2, 4, 3, 4, 3, 1, 2, 1, 1, 1]"
5,"[3.8359418, -1.3124249, -0.40428925, 3.2389076...","[0.7221635, -0.78287965, -0.71455455, 1.264303...","[28.512043, 20.655575, 10.302714, 4.2370133, 4...","[660.92535, 41.216892, 12.066768, 54.13454, 5....","[4.096846, 6.536734, 8.468022, 5.1739287, 6.03...","[45.431393, 70.42315, 86.117676, 45.844795, 51...","[-3.0970938, -3.0680707, -3.0101388, -2.871577...","[-0.82598156, -0.12881298, -1.1559858, 1.30520...","[2.90039100119, -2.68969994418, -2.83425808785...","[1.28182690388, 1.51568250443, -1.05729563163,...","[20.7636454569, 19.4642880689, 19.305490961, 1...","[4, 3, 3, 2, 2, 3, 2, 1, 1, 1, 1, 1]"
6,"[3.3157146, -1.8357779, -1.5356323, -0.1992588...","[-1.5583153, 0.34432852, 0.9947124, 3.1062744,...","[131.41542, 74.273315, 23.9023, 14.9

In [19]:
def deltar(df):
    fulldf['deta']=fulldf['ak4jet_eta']-fulldf['genjet_eta']
    fulldf['dphi']=np.abs(fulldf['ak4jet_phi']-fulldf['genjet_phi'])
    sel=fulldf['dphi']>np.pi
    fulldf['dphi']-=sel*(2*np.pi)
    return(np.sqrt(fulldf['dphi']*fulldf['dphi']+fulldf['deta']*fulldf['deta']))
    
def matching(event):
    if event.matches.sum()==0:
        return event.ak4jet_pt==event.ak4jet_pt.max()
    else:
        cond_a = event.matches==True
        cond_b = event.ak4jet_pt==event[cond_a].ak4jet_pt.max()
        return (cond_a&cond_b)